In [1]:
import xarray as xr
import numpy as np
from affine import Affine
import matplotlib.pyplot as plt

In [28]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [78]:
ds = xr.open_dataset('/home/liyan/Downloads/gsipL2_goes13_GEDISK_2014153_1145.nc.gz',decode_cf=True)[['cloud_mask','pixel_latitude','pixel_longitude']]

/home/liyan/miniconda3/envs/myenv3/lib/python3.6/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'sensor_zenith' has multiple fill values {-999.0, 32767}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/home/liyan/miniconda3/envs/myenv3/lib/python3.6/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'solar_zenith' has multiple fill values {-999.0, 32767}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/home/liyan/miniconda3/envs/myenv3/lib/python3.6/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'relative_azimuth' has multiple fill values {-999.0, 32767}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/home/liyan/miniconda3/envs/myenv3/lib/python3.6/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'reflectance_0.64_mc' has multiple fill values {-999.0, 32767}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/home/liyan/miniconda3/envs/myen

In [77]:
# DOY 121 is 1st May
# Load GOES V3 data and convert to 0.05 degree
# fn is file name 
def load_to_globe(fn):
    ds2 = xr.open_dataset(fn,decode_cf=True)[['cloud_mask','pixel_latitude','pixel_longitude']]
    df = ds2[['pixel_latitude','pixel_longitude','cloud_mask']].to_dataframe().dropna()
    
    # Define Affine of 0.5 degree
    a = Affine(0.05,0,-180,0,-0.05,90)
    
    # get col and row number
    df['col'], df['row'] = ~a * (df['pixel_longitude'], df['pixel_latitude'])
    
    # need to floor to get integer col and row
    df['col'] = df['col'].apply(np.floor).astype(int)
    df['row'] = df['row'].apply(np.floor).astype(int)

    temp = df.groupby(['row','col']).max().reset_index()
    
    c = np.empty((3600,7200,))
    c[:] = np.nan
    
    c[temp['row'],temp['col']]=temp['cloud_mask']
    return c

In [74]:
cloud = load_to_globe('/home/liyan/Downloads/gsipL2_goes13_GENHEM_2014121_0045.nc')

In [76]:
plt.imshow(cloud)


In [52]:
plt.imshow(c)
# plt.show()

In [54]:
plt.figure()
plt.imshow(c==0)


In [51]:
# ds2.cloud_mask
plt.figure()
plt.imshow(ds2.cloud_mask.data)
plt.colorbar()